In [ ]:
# loading packages 
import datacube 
import matplotlib.pyplot as plt
from deafrica_tools.dask import create_local_dask_cluster
from deafrica_tools.plotting import rgb, display_map

In [ ]:
# connecting to the datacube 
dc = datacube.Datacube(app='Step2')

In [ ]:
# lazy loading data through dask chunks parameter
lazy_data = dc.load(product='gm_s2_annual',
                   measurements=['blue','green','red','nir'],
                   x=(31.90, 32.00),
                   y=(30.49, 30.40),
                   time=('2021-01-01', '2021-12-31'),
                   dask_chunks={'time':1,'x':500, 'y':500})
# return data
lazy_data

In [ ]:
# visualise the task graph
lazy_data.red.data.visualize()

In [ ]:
# requesting small portion of red band 
extract_from_red = lazy_data.red[:, 100:200, 100:200]
extract_from_red.data.visualize()

In [ ]:
# calcualte NDVI using red and nir bands from array
band_diff = lazy_data.nir - lazy_data.red
band_sum = lazy_data.nir + lazy_data.red

# added ndvi dask array to the lazy_data dataset
lazy_data['ndvi'] = band_diff / band_sum

In [ ]:
# return the dataset 
lazy_data

In [ ]:
lazy_data.ndvi.data.visualize()

In [ ]:
ndvi_compute = lazy_data.ndvi.compute()
ndvi_compute

In [ ]:
# formatting the figure 
fig , axis = plt.subplots(1, 1, figsize=(6, 6))
axis.set_aspect('equal')

ndvi_compute.plot(ax= axis, cmap='RdYlGn', vmin=0, vmax=1)